# Semantic Segmentation with on the fly Data Augmentation


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras_preprocessing.image import ImageDataGenerator
from gm_networks import unet_model
from utils import center_crop

In [3]:
# Load the initial input and output images and expand their dimensions to fit the ImageDataGenerator 
# For inputs: initial_dimensions = (width, height, 3) ---> new_dimensions = (1, width, height, 3)
# For outputs: initial_dimensions = (width, height) ---> new_dimensions = (1, width, height, 1)
x_initial_train = plt.imread('train/101-INPUT.jpg')
y_initial_train = plt.imread('train/101-OUTPUT-GT.png')
x_initial_train = np.expand_dims(x_initial_train, 0) 
y_initial_train = np.expand_dims(y_initial_train, (0, -1)) 

In [4]:
# Model/training hyperparameters
INPUT_SHAPE = (512, 512, 3)
INITIAL_FILTERS = 32
PADDING = 'same'

TRAIN_IMAGES_NUMBER = 500

BATCH_SIZE = 8
EPOCHS = 800
steps_per_epoch = TRAIN_IMAGES_NUMBER // BATCH_SIZE# Define the new size to create images of size (NEW_SIZE, NEW_SIZE), as well as define the number of train and validation images to be created

In [6]:
model = unet_model(input_shape=INPUT_SHAPE, initial_filters=INITIAL_FILTERS, padding=PADDING)


In [7]:
print(model.output_shape[1:3])
print(y_initial_train.shape)
# TODO: center_crop the y_initial_train tensor 

(512, 512)
(1, 6360, 8300, 1)


In [8]:
new_size = INPUT_SHAPE[0]

# Concatenate the respective inputs and outputs to ensure that they will undergo the same crop
concatenated_images_train = np.concatenate([x_initial_train, y_initial_train], -1)

# Initialize the tensors for the training batches
croped_images_train = np.zeros([TRAIN_IMAGES_NUMBER, new_size, new_size, 4])

# Random croping
for i in range(TRAIN_IMAGES_NUMBER):
    
    t = tf.image.random_crop(concatenated_images_train, size=[1, new_size, new_size, 4])
    croped_images_train[i] = t[0]

# Separate the inputs from the outputs to create the final batches
x_train = croped_images_train[:, :, :, :3]
y_train = np.expand_dims(croped_images_train[:, :, :, 3], -1)

if PADDING == 'valid':
    y_train = center_crop(y_train, model.output_shape[1:3])


In [9]:
print(x_train.shape)

(500, 512, 512, 3)


In [10]:
# Create dictionaries with the alterations to be used for the data augmentation operations for inputs and outputs
""""
NOTE: It is important to use exact the same values for the parameters in both dictionaries. 
The reason why we create 2 dictionaries instead of a common one is because  we want to add the 
preprocessing function for the output masks (this function sets all the pixel values of the mask to 0 or 1. 
While the initial ground truth images are binary, after the data augmentation operations, such as rotations and 
shifts, pixels with intermediate values are appeared due to interpolation)
"""
x_datagen_args = dict(
    rotation_range=25, 
    width_shift_range=0.1, 
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect')

y_datagen_args = dict(
    rotation_range=25, 
    width_shift_range=0.1, 
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='reflect', 
    preprocessing_function = lambda x: np.where(x>0, 1, 0).astype(x.dtype))

In [11]:
# Instatiate the generators
x_datagen = ImageDataGenerator(**x_datagen_args)
y_datagen = ImageDataGenerator(**y_datagen_args)

# Setting the same seed number for both generators leads to the same random alterations for both generators
seed = 19
x_datagen.fit(x_train, augment=True, seed=seed)
y_datagen.fit(y_train, augment=True, seed=seed)

x_generator_train = x_datagen.flow(x_train, seed=seed, batch_size=BATCH_SIZE)
y_generator_train = y_datagen.flow(y_train, seed=seed, batch_size=BATCH_SIZE)

train_generator = zip(x_generator_train, y_generator_train)

In [12]:
# Define the model and train
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

history = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=EPOCHS)

Epoch 1/800


2022-02-22 03:22:35.391220: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8301


62/62 [==============================] - 19s 237ms/step - loss: 0.4970 - accuracy: 0.7555
Epoch 2/800
62/62 [==============================] - 16s 234ms/step - loss: 0.3485 - accuracy: 0.8526
Epoch 3/800
62/62 [==============================] - 15s 235ms/step - loss: 0.2872 - accuracy: 0.8822
Epoch 4/800
62/62 [==============================] - 15s 236ms/step - loss: 0.2951 - accuracy: 0.8787
Epoch 5/800
62/62 [==============================] - 15s 235ms/step - loss: 0.2524 - accuracy: 0.8977
Epoch 6/800
62/62 [==============================] - 15s 238ms/step - loss: 0.2414 - accuracy: 0.9051
Epoch 7/800
62/62 [==============================] - 15s 238ms/step - loss: 0.2209 - accuracy: 0.9103
Epoch 8/800
62/62 [==============================] - 15s 238ms/step - loss: 0.2106 - accuracy: 0.9165
Epoch 9/800
62/62 [==============================] - 15s 239ms/step - loss: 0.2036 - accuracy: 0.9183
Epoch 10/800
62/62 [==============================] - 15s 239ms/step - loss: 0.1940 - accuracy

62/62 [==============================] - 15s 238ms/step - loss: 0.0476 - accuracy: 0.9822
Epoch 81/800
62/62 [==============================] - 15s 237ms/step - loss: 0.0583 - accuracy: 0.9778
Epoch 82/800
62/62 [==============================] - 15s 239ms/step - loss: 0.0529 - accuracy: 0.9800
Epoch 83/800
62/62 [==============================] - 15s 239ms/step - loss: 0.0521 - accuracy: 0.9804
Epoch 84/800
62/62 [==============================] - 15s 237ms/step - loss: 0.0450 - accuracy: 0.9826
Epoch 85/800
62/62 [==============================] - 15s 237ms/step - loss: 0.0379 - accuracy: 0.9857
Epoch 86/800
62/62 [==============================] - 15s 238ms/step - loss: 0.0418 - accuracy: 0.9842
Epoch 87/800
62/62 [==============================] - 15s 238ms/step - loss: 0.2440 - accuracy: 0.9008
Epoch 88/800
62/62 [==============================] - 15s 238ms/step - loss: 0.1297 - accuracy: 0.9500
Epoch 89/800
62/62 [==============================] - 15s 238ms/step - loss: 0.0838 - 

62/62 [==============================] - 15s 238ms/step - loss: 0.0311 - accuracy: 0.9878
Epoch 160/800
62/62 [==============================] - 15s 238ms/step - loss: 0.0293 - accuracy: 0.9884
Epoch 161/800
62/62 [==============================] - 15s 239ms/step - loss: 0.0293 - accuracy: 0.9885
Epoch 162/800
62/62 [==============================] - 15s 237ms/step - loss: 0.0490 - accuracy: 0.9818
Epoch 163/800
62/62 [==============================] - 15s 239ms/step - loss: 0.0315 - accuracy: 0.9880
Epoch 164/800
62/62 [==============================] - 15s 239ms/step - loss: 0.0304 - accuracy: 0.9884
Epoch 165/800
62/62 [==============================] - 15s 239ms/step - loss: 0.0296 - accuracy: 0.9884
Epoch 166/800
62/62 [==============================] - 15s 238ms/step - loss: 0.0273 - accuracy: 0.9892
Epoch 167/800
62/62 [==============================] - 15s 237ms/step - loss: 0.0287 - accuracy: 0.9889
Epoch 168/800
62/62 [==============================] - 15s 237ms/step - loss: 

...

62/62 [==============================] - 15s 236ms/step - loss: 0.0193 - accuracy: 0.9919
Epoch 472/800
62/62 [==============================] - 15s 237ms/step - loss: 0.0194 - accuracy: 0.9920
Epoch 473/800
62/62 [==============================] - 15s 239ms/step - loss: 0.0184 - accuracy: 0.9923
Epoch 474/800
62/62 [==============================] - 15s 239ms/step - loss: 0.0188 - accuracy: 0.9921
Epoch 475/800
62/62 [==============================] - 15s 239ms/step - loss: 0.0190 - accuracy: 0.9921
Epoch 476/800
62/62 [==============================] - 15s 239ms/step - loss: 0.0182 - accuracy: 0.9924
Epoch 477/800
62/62 [==============================] - 15s 238ms/step - loss: 0.0181 - accuracy: 0.9924
Epoch 478/800
62/62 [==============================] - 15s 239ms/step - loss: 0.0179 - accuracy: 0.9926
Epoch 479/800
62/62 [==============================] - 15s 236ms/step - loss: 0.0189 - accuracy: 0.9920
Epoch 480/800
62/62 [==============================] - 15s 238ms/step - loss: 

62/62 [==============================] - 15s 235ms/step - loss: 0.0221 - accuracy: 0.9911
Epoch 550/800
62/62 [==============================] - 15s 237ms/step - loss: 0.0210 - accuracy: 0.9915
Epoch 551/800
62/62 [==============================] - 15s 236ms/step - loss: 0.0188 - accuracy: 0.9922
Epoch 552/800
62/62 [==============================] - 15s 237ms/step - loss: 0.0184 - accuracy: 0.9924
Epoch 553/800
62/62 [==============================] - 15s 238ms/step - loss: 0.0183 - accuracy: 0.9924
Epoch 554/800
62/62 [==============================] - 15s 236ms/step - loss: 0.0188 - accuracy: 0.9922
Epoch 555/800
62/62 [==============================] - 15s 237ms/step - loss: 0.0182 - accuracy: 0.9925
Epoch 556/800
62/62 [==============================] - 15s 240ms/step - loss: 0.0183 - accuracy: 0.9924
Epoch 557/800
62/62 [==============================] - 15s 238ms/step - loss: 0.0196 - accuracy: 0.9920
Epoch 558/800
62/62 [==============================] - 15s 239ms/step - loss: 

62/62 [==============================] - 15s 239ms/step - loss: 0.0172 - accuracy: 0.9928
Epoch 628/800
62/62 [==============================] - 15s 239ms/step - loss: 0.0174 - accuracy: 0.9927
Epoch 629/800
62/62 [==============================] - 15s 238ms/step - loss: 0.0170 - accuracy: 0.9929
Epoch 630/800
62/62 [==============================] - 15s 235ms/step - loss: 0.0166 - accuracy: 0.9930
Epoch 631/800
62/62 [==============================] - 15s 240ms/step - loss: 0.0165 - accuracy: 0.9930
Epoch 632/800
62/62 [==============================] - 15s 239ms/step - loss: 0.0170 - accuracy: 0.9929
Epoch 633/800
62/62 [==============================] - 15s 238ms/step - loss: 0.0170 - accuracy: 0.9928
Epoch 634/800
62/62 [==============================] - 15s 240ms/step - loss: 0.0176 - accuracy: 0.9926
Epoch 635/800
62/62 [==============================] - 15s 242ms/step - loss: 0.0166 - accuracy: 0.9930
Epoch 636/800
62/62 [==============================] - 15s 237ms/step - loss: 

62/62 [==============================] - 15s 236ms/step - loss: 0.0161 - accuracy: 0.9932
Epoch 706/800
62/62 [==============================] - 15s 237ms/step - loss: 0.0160 - accuracy: 0.9932
Epoch 707/800
62/62 [==============================] - 15s 238ms/step - loss: 0.0161 - accuracy: 0.9931
Epoch 708/800
62/62 [==============================] - 15s 239ms/step - loss: 0.0161 - accuracy: 0.9932
Epoch 709/800
62/62 [==============================] - 15s 239ms/step - loss: 0.0172 - accuracy: 0.9928
Epoch 710/800
62/62 [==============================] - 15s 239ms/step - loss: 0.0170 - accuracy: 0.9929
Epoch 711/800
62/62 [==============================] - 15s 240ms/step - loss: 0.0165 - accuracy: 0.9930
Epoch 712/800
62/62 [==============================] - 15s 239ms/step - loss: 0.0162 - accuracy: 0.9932
Epoch 713/800
62/62 [==============================] - 15s 238ms/step - loss: 0.0155 - accuracy: 0.9934
Epoch 714/800
62/62 [==============================] - 15s 235ms/step - loss: 

62/62 [==============================] - 15s 238ms/step - loss: 0.0155 - accuracy: 0.9935
Epoch 784/800
62/62 [==============================] - 15s 237ms/step - loss: 0.0163 - accuracy: 0.9931
Epoch 785/800
62/62 [==============================] - 15s 236ms/step - loss: 0.0151 - accuracy: 0.9936
Epoch 786/800
62/62 [==============================] - 15s 239ms/step - loss: 0.0155 - accuracy: 0.9934
Epoch 787/800
62/62 [==============================] - 15s 236ms/step - loss: 0.0143 - accuracy: 0.9939
Epoch 788/800
62/62 [==============================] - 15s 237ms/step - loss: 0.0148 - accuracy: 0.9936
Epoch 789/800
62/62 [==============================] - 15s 237ms/step - loss: 0.0155 - accuracy: 0.9935
Epoch 790/800
62/62 [==============================] - 15s 237ms/step - loss: 0.0151 - accuracy: 0.9935
Epoch 791/800
62/62 [==============================] - 15s 237ms/step - loss: 0.0154 - accuracy: 0.9936
Epoch 792/800
62/62 [==============================] - 15s 239ms/step - loss: 

In [13]:
import os
# save the model and history of the training (loss, accuracy)
model_path = f'models/Unet_inputsize{new_size}_initialfilters{INITIAL_FILTERS}_padding_{PADDING}_epochs{EPOCHS}' #TODO: loss and metric'
model.save(model_path)


2022-02-22 11:31:29.934143: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/Unet_inputsize512_initialfilters32_padding_same_epochs800/assets


In [14]:
np.save(os.path.join(model_path, 'history'), history.history)